In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
from itertools import chain
from sklearn.model_selection import train_test_split
from scipy.sparse import *
from scipy.sparse.linalg import svds
import random

# implicit 라이브러리를 활용한 멜론 노래 추천

데이터 출처  
- [카카오 아레나 Melon Playlist Continuation](https://arena.kakao.com/c/8/data)

### 데이터 불러오기

In [2]:
playlist = pd.read_json('data/train.json')
song_meta = pd.read_json('data/song_meta.json')
playlist.head(5)

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000


---

### 데이터 변환

- `playlists x songs` 테이블을 한번에 만들기에는 희소행렬의 크기가 너무 커서 scr 방식으로 만들어야 한다

In [3]:
# 각 플레이리스트 곡 수 컬럼 생성
playlist['song_cnt'] = playlist['songs'].progress_apply(lambda x: len(x))

# 플레일리스트에 포함된 모든 노래 리스트
all_songs = list(set(chain(*playlist['songs'])))
# 모든 노래 수
N_songs = len(all_songs)

# 0부터 시작하는 id를 노래에 새로 할당하기 위한 dict
song_to_newid = dict(zip(all_songs, range(N_songs)))
newid_to_song = dict( zip(range(N_songs), all_songs)) 

100%|█████████████████████████████████████████████████████████████████████| 115071/115071 [00:00<00:00, 1027144.25it/s]


In [4]:
#기존 곡 id를 새 id로 바꾼 컬럼을 만든다.
playlist['song_newid'] = playlist['songs'].progress_apply(lambda x: [song_to_newid[song] for song in x])

100%|███████████████████████████████████████████████████████████████████████| 115071/115071 [00:01<00:00, 69767.07it/s]


---

**csr sparse matrix 만들기**

In [5]:
row = np.repeat(range(len(playlist)), playlist['song_cnt'])
col =  np.array(np.concatenate(playlist['song_newid']), dtype = np.int64)
data = np.ones(col.shape[0])

ply_song_table = csr_matrix((data, (row,col) ))
display(ply_song_table)

<115071x615142 sparse matrix of type '<class 'numpy.float64'>'
	with 5285871 stored elements in Compressed Sparse Row format>

- 현재 데이터는 유저의 선호도를 직접적으로 나타낸 평점 데이터가 아니라  
  노래의 포함 여부만을 `0, 1`로 표현한 것이다.

>implicit data 추천에 효과적인 ALS 협업 필터링 사용

# implicit 라이브러리
- [documentation](https://implicit.readthedocs.io/en/latest/quickstart.html)
- [collaborative filtering implicit data](http://yifanhu.net/PUB/cf.pdf) 논문을 구현한 라이브러리
- ALS 알고리즘 사용

In [ ]:
!pip install implicit==0.4.8

In [10]:
from implicit.als import AlternatingLeastSquares as ALS

In [11]:
# 모델링
# 논문에서 가장 좋다고 한 factor 수 200 사용.
# 데이터 크기를 고려하여 iter 수를 기본값의 두배인 30 설정.
als_model = ALS(factors=200, regularization=0.1, iterations = 30)
iterations=als_model.fit(ply_song_table.T * 10)

  0%|          | 0/30 [00:00<?, ?it/s]

### 샘플유저 노래 추천

- 국악연주곡들을 모은 플레이리스트

In [22]:
user_id =56481

display(playlist.iloc[[user_id]])

# 플레이리스트 수록곡
display(song_meta.iloc[playlist.iloc[user_id]['songs']][['song_name','artist_name_basket']])

,tags,id,plylst_title,songs,like_cnt,updt_date,song_cnt,song_newid
56481,"[산책, 드라이브, 여행]",22181,편안하게 들을 수있는 노래,"[429229, 595710, 2318, 54994, 610337, 666130, ...",11,2013-06-24 17:04:08.000,61,"[373098, 517633, 1986, 47633, 530321, 578723, ..."


,song_name,artist_name_basket
429229,그냥니생각이나,[정기고]
595710,들리나요,[태연 (TAEYEON)]
2318,소녀,[이문세]
54994,사랑 첫 느낌,[에일리]
610337,사랑은 이렇게,[케이윌]
...,...,...
199134,거기서거기 (Without You),[다이나믹 듀오]
294935,동행 (同行) (Duet With 호란),[박기영]
272924,Gone (Feat. 주희 Of 에이트),[다이나믹 듀오]
299019,옛사랑,[이문세]


샘플 플레이리스트는 국악 연주곡을 모아 놓은 플레이리스트이다.

In [26]:
%%time
# user_playlist_id = 92059
rec_songs = als_model.recommend(user_id, ply_song_table, N = 30)
# 추천 받은 노래 확인하기
rec_songs_list = [x for x, _ in rec_songs]
song_meta.iloc[ [newid_to_song[x]  for x in rec_songs_list] ][['song_name','artist_name_basket']]

Wall time: 26 ms


,song_name,artist_name_basket
109926,사랑이 다른 사랑으로 잊혀지네,[하림]
325609,사랑..그 놈,[바비 킴]
374617,자니 (Feat. Dynamic Duo),[프라이머리]
566257,좋아보여 (Feat. 검정치마),[버벌진트]
293236,봄봄봄,[로이킴]
228501,굿모닝 (Feat. 권정열 Of 10cm),[버벌진트]
411339,어때 (Feat. 하림),[긱스 (Geeks)]
317057,눈물샤워 (Feat. 에일리),[배치기]
249594,충분히 예뻐 (feat. 산체스 Of 팬텀),[버벌진트]
333408,"Officially Missing You, Too","[긱스 (Geeks), 소유 (SOYOU)]"


-  추천 예측 결과 사극ost 등 동양풍 연주곡 위주의 결과를 확인할 수 있다.

---

# 평가

## 평가지표
### hit rate  
  - 30곡을 추천하여 일치한 노래가 하나라도 있을 경우 `1`, 아닌경우 `0`으로 한다.

### Precision
  -  `적중한 곡 수 / 추천한 곡 수` 
### recall
  -  `적중한 곡 수 / 테스트 곡 수`

### 평가 방식
- 플레이리스트 중 일부에서 곡의 절반을 숨기고 ALS 행렬분해 적용
- ALS 결과 테이블에서 점수가 높은 곡들과 숨긴 원본 곡을 비교하여 평가 

In [55]:
def hit_rate(rec, answer):
    s1 = set(rec) 
    s2 = set(answer)
    if len(s1 & s2) >= 1:
        return 1
    else:
        return 0

def precision(rec, answer):
    s1 = set(rec)
    s2 = set(answer)
    return len(( s1 & s2 )) / len(rec)

def recall(rec, answer):
    s1 = set(rec)
    s2 = set(answer)
    return len(( s1 & s2 )) / len(answer)

## 테스트 데이터

In [29]:
# 노래의 절반을 없앤 컬럼 추가
playlist['song_train'] = playlist['song_newid'].progress_apply(lambda x : x[:(len(x)//2)])
playlist['song_test'] = playlist['song_newid'].progress_apply(lambda x : x[(len(x)//2):])

100%|██████████████████████████████████████████████████████████████████████| 115071/115071 [00:00<00:00, 395432.17it/s]


In [32]:
# train test 인덱스 분리
train_df, test_df = train_test_split(playlist, test_size = 0.1, shuffle = True)

train / test를 분리하더라도 협업테이블을 적용할 테이블의 전체 유저수는 유지되어야 한다. (헙업필터링은 존재하는 유저에 대해서만 추천 가능함)  
`train_df` idx를 가진 플레이리스트는 원본 곡 리스트,  
`test_df` idx를 가진 플레이리스트는 원본 곡의 절반만을 제공한다.  

In [33]:
train_final = pd.DataFrame(train_df['song_newid'].append(test_df['song_train']), columns = ['song_newid'])
train_final['song_cnt'] = train_final['song_newid'].map(lambda x : len(x))
all_train_songs = list(set(chain(*train_final['song_newid'])))
n_songs = len(all_songs)

In [34]:
# 섞인 인덱스 정렬
train_final = train_final.sort_index()

In [35]:
# 검증용 테이블로 희소행렬 생성

row = np.repeat(range(len(train_final)), train_final['song_cnt'])
col =  np.array(np.concatenate(train_final['song_newid']), dtype = np.int64)
data = np.ones(col.shape[0])

train_ply_song_table = csr_matrix((data, (row,col)))

In [16]:
# 검증용 테이블로 ALS 협업필터링 수행

from implicit.als import AlternatingLeastSquares as ALS
als_model = ALS(factors=200, regularization=0.1 ) # 잠재요인 수 200
als_model.fit(train_ply_song_table.T * 40)

  0%|          | 0/15 [00:00<?, ?it/s]

test_df idx를 가진 플레이리스트들의 30곡 추천을 받고  
감추었던 실제 곡 리스트와 비교한다.

In [36]:
# test index 모음
test_index_list = test_df.index
answer_list = test_df['song_test'].tolist()

#예측하기.
rec_list = []
for playlist_id in tqdm(test_index_list):
    rec_songs = als_model.recommend(playlist_id, train_ply_song_table, N = 30)
    rec_songs_list = [x for x, _ in rec_songs]
    rec_list.append(rec_songs_list)

100%|████████████████████████████████████████████████████████████████████████████| 11508/11508 [04:19<00:00, 44.42it/s]


In [54]:
hit_rate_ = 0
precision_ = 0
recall_ = 0
for rec, answer in tqdm(zip(rec_list, answer_list), total = len(test_index_list)):
    hit_rate_ += hit_rate(rec, answer)
    precision_ += precision(rec, answer)
    recall_ += recall(rec, answer)

hit = hit_rate_/len(test_index_list)
prec  = precision_/len(test_index_list)
rec = recall_/len(test_index_list)
f1_score =  2 * ((precision *  recall) / (precision + recall) )

print('hit rate')
print(hit_rate)

print('precision')
print(precision)

print('recall')
print(recall)

print('f1 score')
print(f1_score)

100%|█████████████████████████████████████████████████████████████████████████| 11508/11508 [00:00<00:00, 70595.60it/s]

hit rate
0.8147375738616615
precision
0.1587996755879953
recall
0.23120357432039962
f1 score
0.18828075204751835


### 무작위 유저의 곡 추천 예측 확인

In [59]:
sample_user = random.sample(list(test_index_list), 1)[0]
sample_user_test = playlist.iloc[sample_user]['song_test']
sample_user_rec = als_model.recommend(sample_user, train_ply_song_table, N = 30)
sample_user_rec = [x for x, _ in sample_user_rec]
display("원래 가진 노래", song_meta.iloc[ [newid_to_song[x]  for x in sample_user_test] ][['song_name','artist_name_basket']][:15])
display("추천된 노래", song_meta.iloc[ [newid_to_song[x]  for x in sample_user_rec] ][['song_name','artist_name_basket']][:15])

'원래 가진 노래'

,song_name,artist_name_basket
77547,애월,[문문 (MoonMoon)]
131295,비행운,[문문 (MoonMoon)]
368044,물감,[문문 (MoonMoon)]
123837,좋아했나봐 (Feat. 매드클라운),[마인드유]
115359,얘지,[HOONIA]
289427,Tomorrow,[그리즐리 (Grizzly)]
461212,말과 말 사이,[D.no]
692514,그 자리에 있어줘,[iamnot (아이엠낫)]
387781,안 미안해 (I`M NOT SORRY),[이형은]
605730,Fall In Love,[오왠 (O.WHEN)]


'추천된 노래'

,song_name,artist_name_basket
131295,비행운,[문문 (MoonMoon)]
233166,선을 그어 주던가,[1415]
368044,물감,[문문 (MoonMoon)]
478754,선물,[멜로망스]
700348,"Walking in the Moonlight (Feat. 다원, Lazier)",[서교동의 밤]
300087,Madeleine Love,[CHEEZE (치즈)]
576186,좋아해 (bye),[CHEEZE (치즈)]
205247,밤이 되니까,[펀치 (Punch)]
683520,어떻게 생각해,[CHEEZE (치즈)]
625996,Autumn Breeze (Feat. Rachel Lim),[JIDA (지다)]


>**평가 지표상 점수는 굉장히 낮지만 원본 플레이리스트의 곡들과 유사한 곡들이 추천되는 것을 확인 할 수 있었다.**